In [ ]:
#import data from a csv file and store in a dataframe
import pandas as pd
df = pd.read_csv('../data/Hotel_Reviews.csv')



df['Negative_Review'].isnull().sum(), df['Positive_Review'].isnull().sum()
# Clean the reviews from the dataset
#removing invalid ones, e.g. empty reviews, everything is good/bad and reviews with less than 10 characters
# output the df with two new columns: 'Negative_clean' and 'Positive_clean'

# process the Negative_Review column
# strip() to remove leading and trailing whitespaces for one column of df
import numpy as np

#change data type to string for the column NegativeReview
df['NegativeReview'] = df['Negative_Review'].astype(str)

#show data type for the column NegativeReview
print(type(df['NegativeReview'][0]))

df['Negative_clean'] = df['NegativeReview'].apply(lambda x: x.strip())

# if the text has less than 10 characters, replace the text with 'nothing' for the above column
df.loc[df['Negative_clean'].str.len() < 10, 'Negative_clean'] = ' '

# if the text has 'nothing' or 'everything' or 'anything' str in it regardless lower case, replace the text with 'nothing' for the above column
df.loc[df['Negative_clean'].str.contains('nothing|everything|anything|No Negative', case=False), 'Negative_clean'] = ' '

# do the same process for Positive_Review column
df['Positive_clean'] = df['Positive_Review'].apply(lambda x: x.strip())

# if the text has less than 10 characters, replace the text with 'nothing' for the above column
df.loc[df['Positive_clean'].str.len() < 10, 'Positive_clean'] = ' '

df['Positive_clean'].value_counts().head(30)

# if the text has 'nothing' or 'everything' or 'No Positive' str in it regardless lower case, replace the text with 'nothing' for the above column
df.loc[df['Positive_clean'].str.contains('nothing|everything|No Positive', case=False), 'Positive_clean'] = ' '
df['Positive_clean'].value_counts().head(30)


In [ ]:
# check ' ' value for the new columns
neg_invalid_count = df['Negative_clean'][df['Negative_clean']==" "].shape[0]
pos_invalid_count = df['Positive_clean'][df['Positive_clean']==" "].shape[0]
pos_invalid_count/len(df), neg_invalid_count/len(df)

In [ ]:
# merger two 'Negative_clean', 'Positive_clean' columns and save to a new column 'Reviews', and save to a new column 'Reviews_clean'
df['Reviews_clean'] = df['Negative_clean'] + df['Positive_clean']
df['Reviews_clean']

In [ ]:
df['Reviews_clean'].value_counts().head(10)



In [ ]:
# if the value is ' ', drop the row
df_new = df[df['Reviews_clean'] != '  '][['Reviews_clean']]
df_new.shape

In [ ]:
df_new.value_counts().head(30)

In [ ]:
type(df_new)
# the data is cleaned and ready for further analysis

In [ ]:
# get the length of each string in column 'Reviews_clean'
df_new['length'] = df_new['Reviews_clean'].str.len()

df_new

In [ ]:
#get the mean of the length, but should check the word count instead, can do it 
#by using nltk library
df_new['length'].mean(), df_new['length'].median(), df_new['length'].max(), df_new['length'].min()

In [228]:
seed_keywords = [
    "Bar",
    "Bathroom",
    "Bed",
    "Breakfast",
    "Bus",
    "Checkin",
    "Clean",
    "Cleanliness",
    "Coffee",
    "Comfort",
    "Dinner",
    "Facilities",
    "WiFi",
    "Heat",
    "Hot",
    "Location",
    "Luggage",
    "Noise",
    "Pool",
    "Quiet",
    "Room",
    "Shower",
    "Spa",
    "Staff",
    "Storage",
    "Suite",
    "Toilet",
    "Wifi"
]


In [236]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

# function to get the keywords from the text using KeyBERT
def keywords_extract(text):
    sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
    kw_model = KeyBERT(model=sentence_model)
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', top_n=100, seed_keywords=seed_keywords)
    return keywords

doc = df_new[['Reviews_clean']].head(10)



In [237]:
doc

,Reviews_clean
0,I am so angry that i made this post available ...
1,No real complaints the hotel was great great ...
2,Rooms are nice but for elderly a bit difficult...
3,My room was dirty and I was afraid to walk bar...
4,You When I booked with your company on line yo...
5,Backyard of the hotel is total mess shouldn t ...
6,Cleaner did not change our sheet and duvet eve...
7,Good location Set in a lovely park friendly s...
8,Even though the pictures show very clean rooms...
9,The aircondition makes so much noise and its h...


In [238]:

doc_list = doc['Reviews_clean'].tolist()
doc_list


['I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that i booked a duplex btw it costs the same as a simple double but got way more volume due to the high ceiling was offered a room but only the next day SO i had to check out the next day before 11 o clock in order to get the room i waned to Not the best way to begin y

In [240]:
keywords = keywords_extract(doc_list)
keywords

[[('hotel', 0.5655),
  ('room', 0.5293),
  ('booking', 0.4849),
  ('ceiling', 0.3716),
  ('booked', 0.3678),
  ('ceilings', 0.333),
  ('place', 0.3258),
  ('floor', 0.3034),
  ('holiday', 0.3033),
  ('fridge', 0.296),
  ('belongings', 0.2938),
  ('planing', 0.2917),
  ('vent', 0.2893),
  ('duplex', 0.2769),
  ('evening', 0.2731),
  ('reserved', 0.2705),
  ('nights', 0.26),
  ('trips', 0.2594),
  ('possible', 0.2464),
  ('clock', 0.2373),
  ('closed', 0.232),
  ('arrival', 0.2317),
  ('smell', 0.2281),
  ('order', 0.2244),
  ('2nd', 0.2239),
  ('stayed', 0.2234),
  ('window', 0.2203),
  ('mistake', 0.2198),
  ('volume', 0.2181),
  ('contained', 0.2157),
  ('mind', 0.2132),
  ('able', 0.2132),
  ('park', 0.2117),
  ('garden', 0.2055),
  ('internal', 0.2047),
  ('simple', 0.2046),
  ('mail', 0.2038),
  ('begin', 0.1977),
  ('determine', 0.1975),
  ('asleep', 0.1972),
  ('check', 0.1971),
  ('change', 0.1913),
  ('sound', 0.1906),
  ('sort', 0.188),
  ('double', 0.1857),
  ('btw', 0.1832),

In [ ]:
test_df = df_new.tail(1000) 

In [231]:
# convert a column to a list
list_A = test_df['Reviews_clean'].tolist()
list_A

[' Good location large hotel with good facilities',
 'In this day and age free wifi is a must Would have been nice if pool access includedGreat location and friendly staff',
 'The room smelt musty ',
 'The unexpected deposit asked of us that wasnt on booking confirmation Staff attenpting to enter our room on 2 occasions early morning ',
 'The staff what we paid for what we received having to pay to get to the airportVery close to the airport',
 'Room key was a bit dodgy on two separate occasions but they upgraded us to another room immediatelyExcellent customer service',
 ' Worst experience ever',
 'Rooms very compact ',
 'Device facilities ',
 'you have to pay for the leisure centre ',
 'The building work going on the housekeeping kept coming into our roomStaff were friendly',
 'Really crappy toaster One none industrial toaster taking 4 slices Absolute jokeInformative cheerful staff',
 ' Location was convenient and it had been upgraded since my last stay most notably with a c',
 'We w

In [ ]:
# get the list of keywords for each the review in the dataframe, append to a big list

test_df['keywords'] = test_df['Reviews_clean'].apply(lambda x: keywords_extract(x))


In [ ]:
test_df

In [ ]:
# flatten column 'keywords' to get a list of keywords
list_list = test_df['keywords'].tolist()

In [ ]:
# flatten the list of list
one_list = [item for sublist in list_list for item in sublist]
one_list


In [ ]:
# convert the list to a DataFrame
keyword_df = pd.DataFrame(one_list, columns=['keyword', 'score'])
keyword_df

In [ ]:
# get the top 10 keywords by groupby keyword and get the sum of the score
top_keywords = keyword_df.groupby('keyword').sum().sort_values(by='score', ascending=False)




In [227]:
df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng', 'NegativeReview', 'Negative_clean',
       'Positive_clean', 'Reviews_clean'],
      dtype='object')